In [1]:
from student.tools import *
from student.agent import *

In [2]:
ag = MemoryAgent()
tool_choice = "auto"
# res = ag.run("Add (stimulus 'test', content 'this is a test') to the memory", parse=None, tool_choice=tool_choice)

In [ ]:
ag.run("use the add memory tool for a test run", parse=None)
ag.memory.memory

In [ ]:
# print('I can choose from two main tools: \n\n1. **functions.add** - To add new memory items.\n2. **functions.modify** - To modify existing memory items.\n3. **functions.recall** - To recall memory items based on specified keys and sensitivity.\n\nAdditionally, I can use **multi_tool_use.parallel** to execute multiple tools simultaneously if needed. \n\nIf you have any specific tasks in mind, feel free to let me know!')
# print('Building a memory system for a large language model (LLM) agent involves several key components and considerations. Here’s a general outline of how to approach this:\n\n### 1. Define Memory Types\n   - **Short-term Memory**: Temporary storage for recent interactions or data that the agent can access quickly.\n   - **Long-term Memory**: Persistent storage for knowledge and experiences that the agent can recall over time.\n\n### 2. Memory Structure\n   - **Key-Value Pairs**: Store information as pairs where a key (stimulus) is associated with a value (content).\n   - **Hierarchical Structure**: Organize memories in a hierarchy or categories for easier retrieval.\n\n### 3. Memory Operations\n   - **Add Memory**: Function to store new information.\n   - **Modify Memory**: Function to update existing information.\n   - **Recall Memory**: Function to retrieve information based on specific keys or queries.\n\n### 4. Sensitivity and Context\n   - Implement sensitivity levels to control how closely the recalled memory should match the query.\n   - Contextual awareness to ensure the agent retrieves relevant memories based on the current conversation or task.\n\n### 5. Memory Management\n   - **Expiration Policy**: Define how long memories should be retained before they are considered outdated and removed.\n   - **Conflict Resolution**: Determine how to handle conflicting memories or updates.\n\n### 6. Integration with LLM\n   - Ensure that the memory system is seamlessly integrated with the LLM’s processing pipeline, allowing it to access and utilize memories during interactions.\n\n### 7. Testing and Iteration\n   - Continuously test the memory system with various scenarios to ensure it performs as expected.\n   - Iterate on the design based on feedback and performance metrics.\n\n### 8. Ethical Considerations\n   - Ensure that the memory system respects user privacy and data protection regulations.\n   - Implement mechanisms for users to view and delete their stored memories if desired.\n\n### Example Implementation\n- Use a database or in-memory data structure to store memories.\n- Create APIs for adding, modifying, and recalling memories.\n- Use natural language processing techniques to parse and understand user inputs for effective memory management.\n\nBy following these guidelines, you can create a robust memory system that enhances the capabilities of an LLM agent, allowing it to learn from interactions and provide more personalized responses.')

In [3]:
m = Memory()

In [4]:
d = {'keys':["a"], "content": "test a"}
m.add_from_dict(d)

In [5]:
n = MemoryNode(content="test a", keys=["a"])
m.add(n)
n = MemoryNode(content="test b", keys=["b"])
m.add(n)
m.memory

In [6]:
for node in m.memory:
    print(node.keys)

['a']
['a']
['b']


In [ ]:

scores = []
nodes = m.get_nodes()

for node in nodes:
    scores.append(node.get_score(["a"]))
scores

In [8]:
from student.bm25_indexing import get_bm25_score
get_bm25_score(["a", "b"], ["a"])

[0.0]

In [9]:
m.search(queries=["bc"])